In [1]:
import os
import sys

# Ensure no W&B logging will be performed
sys.argv = "main.py -log tb -name tst -reset 1 -lm.eval.enable 0 -log tb -batch_size 20 -restore /cm/shared/anonymous/moeut_training_code/tmp/smoe-softmax-10000.pth".split(" ")
# sys.argv = "main.py -log tb -name tst -reset 1 -lm.eval.enable 0 -log tb -batch_size 20 -restore paper/moe_universal/checkpoints/plvywltl/model.ckpt".split(" ")

# Pretend we are in the main directory
os.chdir("../../")

In [2]:
from main import initialize
import torch
import torch.nn.functional as F
from layers import MoE

print(MoE)

/cm/archive/anonymous/miniconda3/envs/moeut/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-02 16:35:10,519] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/cm/archive/anonymous/miniconda3/envs/moeut/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/cm/archive/anonymous/miniconda3/envs/moeut/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


<class 'layers.moe_layer.MoE'>


In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter

plt.rcParams['text.usetex'] = False #Let TeX do the typsetting
plt.rcParams['text.latex.preamble'] = '\\usepackage{sansmath}\n\\sansmath' #Force sans-serif math mode (for axes labels)
plt.rcParams['font.family'] = 'DejaVu Sans' # Change from 'Arial' to 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial' # Choose a nice font here

plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

In [4]:
helper, task = initialize()
task.create_data_fetcher()

orig_run_model_valid = task.run_model_validation

Restoring: /cm/shared/anonymous/moeut_training_code/tmp/smoe-softmax-10000.pth...
No distributed environment detected
Resetting training state...
Port already used: 7000
Port already used: 7001
Found tensorboard in /cm/archive/anonymous/miniconda3/envs/moeut/bin/tensorboard
Starting Tensorboard server on 7002
Done.
Available GPUs: 8
SlimPajama: Generating map...
Map done.
SlimPajama: Loaded tokenizer.
SlimPajama: 58345 chunks missing
Token limit reached. No need to tokenize more.
SlimPajama: Limiting to first 164 chunks because limited to 2050048000 tokens
SlimPajama: Loaded tokenizer.
SlimPajama: 31345 chunks missing
Token limit reached. No need to tokenize more.
SlimPajama: Limiting to first 59 chunks because limited to 1032192 tokens
Weight info:
  n_model_weights: 149733376
  n_attention_weights: 10747904
  n_non_attnetion_weights: 138985472
  attention_precent: 0.0717802823065981
Training in bfloat16...
Total number of model parameters: 157934400
Loading state
Loading run_invarian

/cm/shared/anonymous/moeut_training_code/tasks/lm_base.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor(res, dtype=torch.long, device=self.helper.device).T


In [5]:
nexp = task.helper.args.moe.n_experts
ntok = task.helper.args.sentencepiece.n_pieces
# ngrp = task.helper.args.transformer.universal.group_size
nlayers = task.helper.args.transformer.encoder_n_layers
context = task.helper.args.lm.unroll
k = task.helper.args.pkm.n_heads
bsz = task.helper.args.batch_size
ngrp = 16

token_counts = 0

cnt = 0
simmap = torch.zeros(ngrp, nlayers // ngrp, nlayers // ngrp)

thissel = torch.zeros(ngrp, nlayers // ngrp, bsz, context, k, dtype=torch.long)

In [6]:
ngrp

16

In [7]:
global this_data

def run_model_validation(self, data):
    global token_counts
    global this_data
    global cnt
    global simmap

    token_counts = token_counts + F.one_hot(data["data"].flatten().long(), ntok).sum(0)

    this_data = data

    thissel.zero_()

    res = orig_run_model_valid(data)

    ohsel = F.one_hot(thissel, nexp).sum(-2)
    ohsel = (ohsel.flatten(2,3).permute(0, 2, 1, 3) > 0).float()

    #shape: ngrp, bsz*context, nlayer, nexp
    overlap = torch.einsum("nglk,ngok->nglo", ohsel, ohsel)
    norm = torch.maximum(ohsel.unsqueeze(-3), ohsel.unsqueeze(-2)).sum(-1)
    simcnt = (overlap / norm).sum(1)
    cnt += ohsel.shape[1]
    simmap += simcnt

    return res

task.run_model_validation = run_model_validation.__get__(task)

In [8]:
task

In [9]:
id_map = {}

def patch_module(module):

    myid = id(module)
    if myid in id_map:
        return

    gid = len(id_map)
    id_map[myid] = gid

    # sel_val, sel_index = self.topk(

    def new_topk(self, *args, **kwargs):
        gid = id_map[id(self)]

        sel_val, sel_index = MoE.topk(self, *args, **kwargs)

        thissel[gid, self.layer//ngrp] = sel_index

        return sel_val, sel_index


    module.topk = new_topk.__get__(module)


In [10]:
for m in task.model.modules():
    if isinstance(m, MoE):
        patch_module(m)

In [11]:
task.validate()

Starting validation on val...


  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (20) must match the existing size (10) at non-singleton dimension 0.  Target sizes: [20, 1024, 8].  Tensor sizes: [10, 1024, 8]

: 

In [ ]:
avg = simmap/cnt

In [ ]:
gid = 0
fig, ax=plt.subplots(figsize=(2.4,2))
plt.imshow(avg[0], cmap='viridis', aspect='auto')
plt.colorbar()
plt.xticks([a*2 + gid for a in range(nlayers // ngrp // 2 + 1)], [str(a*4 + 1 + gid) for a in range(nlayers // ngrp // 2 + 1)])
plt.yticks([a*2 + gid for a in range(nlayers // ngrp // 2 + 1)], [str(a*4 + 1 + gid) for a in range(nlayers // ngrp // 2+1)])
plt.xlabel("Layer")
plt.ylabel("Layer")
plt.tight_layout()
plt.savefig("paper/moe_universal/layer_similarity.pdf", bbox_inches='tight')